In [1]:
# Importando bibliotecas
import datetime as dt
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 100)

C:\Users\Caio Paulino\AppData\Local\Temp\ipykernel_13368\2777480918.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Classe responsável por armazenar os dados de folhosas e inmet por csv
class AllDados:
    def __init__(self, folhosas_csv, inmet_sp_csv, inmet_sorocaba_csv):
        # Armazenando os dados da tabela csv em um DataFrame pandas
        self.folhosas = pd.read_csv(folhosas_csv)
        self.inmet_sp = pd.read_csv(inmet_sp_csv)
        self.inmet_sorocaba = pd.read_csv(inmet_sorocaba_csv)

In [3]:
# Classe responsável por processar dados
class ProcessadorDados:    
    # Método responsável por limpar outliers
    def clean_outliers(df):
        df = df[(df != -9999).all(axis=1)]
        df.loc[df.eq(-9999).any(axis=1)]
        
        return df

    # Método responsável por agrupar dados por Data para descobrir a média diária de cada coluna
    def daily_groupby(df):
        df = df.groupby(['Data']).mean()
        df.reset_index(inplace=True)

        return df
    
    # Método responsável por extrair as datas de catálogo de produtos
    def date_extract(df):
        # Selecionando datas únicas
        datas = df['Data'].unique()

        # Encontrando a data anterior a uma semana do primeiro produto
        primeira_data = pd.Timestamp(datas[0]) - dt.timedelta(days=7)

        # Adicionando a primeira data as datas descobertas anteriormente
        datas = pd.Series(datas)
        datas = pd.concat([pd.Series([primeira_data]), datas], ignore_index=True)
        
        return datas

    # Método responsável por agrupar dados pelos intervalos das datas de catálogo descobrir as médias de cada coluna
    def weekly_groupby(inmet, datas):
        # Criando um DataFrame para armazenar os dados do inmet
        inmet_weekly = pd.DataFrame()

        # Foreach para percorrer as datas identificadas e realizar os processamentos de dados
        for i in range(len(datas) - 1):
            intervalo_datas = inmet[(inmet['Data'] >= datas[i]) & (inmet['Data'] <= datas[i + 1])]

            pivot = intervalo_datas[
                ['Média Precipitação Total (mm)', 'Média Pressão Atmosférica (mB)', 'Média Temperatura do Ar (°C)',
                 'Média Umidade Relativa do Ar (%)', 'Média Vento Velocidade (m/s)']].mean()
            
            pivot['Data'] = datas[i + 1]
            
            inmet_weekly = pd.concat([inmet_weekly, pd.DataFrame([pivot])], ignore_index=True)

        return inmet_weekly
    
    # Método responsável por executar os processamentos de dados e gerar um novo inmet
    def process_inmet(folhosas, inmet):
        inmet = ProcessadorDados.clean_outliers(inmet)
        inmet['Data'] = pd.to_datetime(inmet['Data'])

        datas = ProcessadorDados.date_extract(folhosas)
        inmet_processed = ProcessadorDados.weekly_groupby(inmet, datas)

        return inmet_processed
    
    # Método responsável por formatar colunas DataFrame(arredondamento de casas decimais)
    def format_columns(df):
        df['Média Precipitação Total (mm)'] = df['Média Precipitação Total (mm)'].round(4)
        df['Média Pressão Atmosférica (mB)'] = df['Média Pressão Atmosférica (mB)'].round(3)
        df['Média Temperatura do Ar (°C)'] = df['Média Temperatura do Ar (°C)'].round(2)
        df['Média Umidade Relativa do Ar (%)'] = df['Média Umidade Relativa do Ar (%)'].round(2)
        df['Média Vento Velocidade (m/s)'] = df['Média Vento Velocidade (m/s)'].round(2)
        df['Média Umidade Relativa do Ar (%)'] = df['Média Umidade Relativa do Ar (%)'].astype(str) + '%'

        return df
    
    # Método responsável por fundir os dados de folhosas e inmet
    def merge_folhosas_inmet(folhosas, inmet):
        # Merge
        folhosas_inmet = pd.merge(folhosas, inmet)

        # Retorna DataFrame formatado
        return ProcessadorDados.format_columns(folhosas_inmet)

In [4]:
# Classe responsável por armazenar os dados de Mogi
class Mogi():
    def __init__(self, folhosas, inmet_sp):
        self.folhosas = folhosas[folhosas['Região'] == "Mogi das Cruzes"]
        self.inmet = ProcessadorDados.process_inmet(self.folhosas, inmet_sp)
        self.folhosas_inmet = ProcessadorDados.merge_folhosas_inmet(self.folhosas, self.inmet)

In [5]:
# Classe responsável por armazenar os dados de Ibiúna
class Ibiuna():
    def __init__(self, folhosas, inmet_sorocaba):
        self.folhosas = folhosas[folhosas['Região'] == "Ibiúna"]
        self.inmet = ProcessadorDados.process_inmet(self.folhosas, inmet_sorocaba)
        self.folhosas_inmet = ProcessadorDados.merge_folhosas_inmet(self.folhosas, self.inmet)

In [6]:
# Instância de AllDados (Salvando dados de folhosas, inmet_sp e inmet_sorocaba)
all_dados = AllDados('Folhosas.csv', 'INMET\\SP_2013_2023.csv', 'INMET\\SOROCABA_2013_2023.csv')

# Instância de Mogi (Salvando dados de mogi a partir de folhosas e inmet_sp)
mogi = Mogi(all_dados.folhosas, all_dados.inmet_sp)

# Instância de Ibiúna (Salvando dados de ibiuna a partir de folhosas e inmet_sorocaba)
ibiuna = Ibiuna(all_dados.folhosas, all_dados.inmet_sorocaba)

C:\Users\Caio Paulino\AppData\Local\Temp\ipykernel_13368\601548811.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inmet['Data'] = pd.to_datetime(inmet['Data'])
C:\Users\Caio Paulino\AppData\Local\Temp\ipykernel_13368\601548811.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inmet['Data'] = pd.to_datetime(inmet['Data'])


In [7]:
mogi.folhosas_inmet

,Produto,Região,Preço,Dia,Mês,Ano,Data,Média Precipitação Total (mm),Média Pressão Atmosférica (mB),Média Temperatura do Ar (°C),Média Umidade Relativa do Ar (%),Média Vento Velocidade (m/s)
0,Alface Americana,Mogi das Cruzes,"R$ 9,43",5,3,2013,2013-03-05 00:00:00,0.1229,925.893,23.47,69.19%,2.24
1,Alface Crespa,Mogi das Cruzes,"R$ 7,00",5,3,2013,2013-03-05 00:00:00,0.1229,925.893,23.47,69.19%,2.24
2,Alface Lisa,Mogi das Cruzes,"R$ 8,00",5,3,2013,2013-03-05 00:00:00,0.1229,925.893,23.47,69.19%,2.24
3,Alface Americana,Mogi das Cruzes,"R$ 7,67",12,3,2013,2013-03-12 00:00:00,0.5771,924.426,24.23,74.76%,1.89
4,Alface Crespa,Mogi das Cruzes,"R$ 6,00",12,3,2013,2013-03-12 00:00:00,0.5771,924.426,24.23,74.76%,1.89
...,...,...,...,...,...,...,...,...,...,...,...,...
1616,Alface Crespa,Mogi das Cruzes,"R$ 17,83",31,8,2023,2023-08-31 00:00:00,0.1060,928.265,15.78,77.25%,1.84
1617,Alface Lisa,Mogi das Cruzes,"R$ 17,83",31,8,2023,2023-08-31 00:00:00,0.1060,928.265,15.78,77.25%,1.84
1618,Alface Americana,Mogi das Cruzes,"R$ 15,25",8,9,2023,2023-09-08 00:00:00,0.0000,928.829,18.32,63.75%,1.19
1619,Alface Crespa,Mogi das Cruzes,"R$ 17,67",8,9,2023,2023-09-08 00:00:00,0.0000,928.829,18.32,63.75%,1.19


In [10]:
ibiuna.folhosas_inmet

,Produto,Região,Preço,Dia,Mês,Ano,Data,Média Precipitação Total (mm),Média Pressão Atmosférica (mB),Média Temperatura do Ar (°C),Média Umidade Relativa do Ar (%),Média Vento Velocidade (m/s)
0,Alface Americana,Ibiúna,800.0,5,3,2013,2013-03-05 00:00:00,0.1625,945.417,23.49,74.68,2.20
1,Alface Crespa,Ibiúna,550.0,5,3,2013,2013-03-05 00:00:00,0.1625,945.417,23.49,74.68,2.20
2,Alface Lisa,Ibiúna,500.0,5,3,2013,2013-03-05 00:00:00,0.1625,945.417,23.49,74.68,2.20
3,Alface Americana,Ibiúna,400.0,12,3,2013,2013-03-12 00:00:00,0.5365,944.121,23.72,81.38,1.74
4,Alface Crespa,Ibiúna,400.0,12,3,2013,2013-03-12 00:00:00,0.5365,944.121,23.72,81.38,1.74
...,...,...,...,...,...,...,...,...,...,...,...,...
1566,Alface Crespa,Ibiúna,1333.0,31,8,2023,2023-08-31 00:00:00,0.1078,948.198,16.36,74.87,0.32
1567,Alface Lisa,Ibiúna,1333.0,31,8,2023,2023-08-31 00:00:00,0.1078,948.198,16.36,74.87,0.32
1568,Alface Americana,Ibiúna,1367.0,8,9,2023,2023-09-08 00:00:00,0.0000,948.733,18.58,60.96,0.05
1569,Alface Crespa,Ibiúna,1367.0,8,9,2023,2023-09-08 00:00:00,0.0000,948.733,18.58,60.96,0.05


In [8]:
# Convertendo a coluna Preço de string para float
mogi.folhosas_inmet['Preço'] = mogi.folhosas_inmet['Preço'].replace('[R\$\,]', '', regex=True).astype(float)

# Removendo o caractere '%' e convertendo a coluna 'Média Umidade Relativa do Ar (%)' para float
mogi.folhosas_inmet['Média Umidade Relativa do Ar (%)'] = mogi.folhosas_inmet['Média Umidade Relativa do Ar (%)'].str.replace('%', '').astype(float)

# Selecionando apenas as colunas relevantes para calcular a correlação
colunas_corr = ['Preço', 'Média Precipitação Total (mm)', 'Média Pressão Atmosférica (mB)', 
                'Média Temperatura do Ar (°C)', 'Média Umidade Relativa do Ar (%)', 'Média Vento Velocidade (m/s)']

df_corr = mogi.folhosas_inmet[colunas_corr]

# Calculando a correlação
correlacoes = df_corr.corr()

# Exibindo a matriz de correlação
print(correlacoes)

                                     Preço  Média Precipitação Total (mm)  \
Preço                             1.000000                       0.153833   
Média Precipitação Total (mm)     0.153833                       1.000000   
Média Pressão Atmosférica (mB)   -0.118945                      -0.482392   
Média Temperatura do Ar (°C)      0.177066                       0.313103   
Média Umidade Relativa do Ar (%)  0.056873                       0.440230   
Média Vento Velocidade (m/s)     -0.177602                       0.045275   

                                  Média Pressão Atmosférica (mB)  \
Preço                                                  -0.118945   
Média Precipitação Total (mm)                          -0.482392   
Média Pressão Atmosférica (mB)                          1.000000   
Média Temperatura do Ar (°C)                           -0.646923   
Média Umidade Relativa do Ar (%)                       -0.172026   
Média Vento Velocidade (m/s)                        

In [9]:
# Convertendo a coluna Preço de string para float
ibiuna.folhosas_inmet['Preço'] = ibiuna.folhosas_inmet['Preço'].replace('[R\$\,]', '', regex=True).astype(float)

# Removendo o caractere '%' e convertendo a coluna 'Média Umidade Relativa do Ar (%)' para float
ibiuna.folhosas_inmet['Média Umidade Relativa do Ar (%)'] = ibiuna.folhosas_inmet['Média Umidade Relativa do Ar (%)'].str.replace('%', '').astype(float)

# Selecionando apenas as colunas relevantes para calcular a correlação
colunas_corr = ['Preço', 'Média Precipitação Total (mm)', 'Média Pressão Atmosférica (mB)', 
                'Média Temperatura do Ar (°C)', 'Média Umidade Relativa do Ar (%)', 'Média Vento Velocidade (m/s)']

df_corr = ibiuna.folhosas_inmet[colunas_corr]

# Calculando a correlação
correlacoes = df_corr.corr()

# Exibindo a matriz de correlação
print(correlacoes)

                                     Preço  Média Precipitação Total (mm)  \
Preço                             1.000000                       0.046157   
Média Precipitação Total (mm)     0.046157                       1.000000   
Média Pressão Atmosférica (mB)   -0.079340                      -0.457069   
Média Temperatura do Ar (°C)      0.169994                       0.203412   
Média Umidade Relativa do Ar (%)  0.022527                       0.536192   
Média Vento Velocidade (m/s)     -0.389416                      -0.058851   

                                  Média Pressão Atmosférica (mB)  \
Preço                                                  -0.079340   
Média Precipitação Total (mm)                          -0.457069   
Média Pressão Atmosférica (mB)                          1.000000   
Média Temperatura do Ar (°C)                           -0.667633   
Média Umidade Relativa do Ar (%)                       -0.224849   
Média Vento Velocidade (m/s)                        